# 7. Data Enrichment 

Given the tags present in the data entries with the field 'malware'.
If the field 'malware' is NaN, i have use both the TAGS and the MALWARES dataset of the corresponding data entry to predict the value. 

1. I have applied similarity measures ( jaro wrinkler similarity index ) to the malwares and group the similiar malwares ( denoting the ALIAS dataset )

2. I have applied similarity measures ( jaro wrinkler similarity index ) between tags and the resulting ALIAS dataset

N.B. I note that in tags can be introduced multiple malwares so i need to find a criteria to choose or to consider the relationship ( entry, malware ) as 1:N


In [1]:
from datetime import datetime
import os
import time
import pandas as pd 
import requests
import urllib3
import json
import sys
import numpy as np
import math

pd.options.display.max_colwidth = 1000

In [2]:
entries_csv = r'C:\Users\leona\OneDrive\Desktop\Tesi\Pipeline\Datasets\Pipeline\IocSegnalations\IntegratedDataset\PostDataCleaning\Entries.csv'
malwares_csv = r'C:\Users\leona\OneDrive\Desktop\Tesi\Pipeline\Datasets\Pipeline\IocSegnalations\IntegratedDataset\PostDataCleaning\Malwares.csv'
alias_csv = r'C:\Users\leona\OneDrive\Desktop\Tesi\Pipeline\Datasets\Pipeline\IocSegnalations\IntegratedDataset\PostDataCleaning\Alias.csv'
tags_csv = r'C:\Users\leona\OneDrive\Desktop\Tesi\Pipeline\Datasets\Pipeline\IocSegnalations\IntegratedDataset\PostDataCleaning\Tags.csv'

## Entries

In [3]:
entries_df = pd.read_csv(entries_csv, low_memory = False)
entries_df = entries_df.drop('Unnamed: 0', axis = 1)
entries_df

,ID_ENTRY,source,ioc,ioc_type,threat_type,malware,first_seen,last_seen,reporter,reference
0,0,Feodo Tracker,89.101.97.139:443,ip:port,botnet,qakbot,2021-09-29,2023-08-25,Feodo Tracker blocklist,https://feodotracker.abuse.ch/
1,1,Feodo Tracker,41.228.22.180:443,ip:port,botnet,qakbot,2021-09-29,2023-08-25,Feodo Tracker blocklist,https://feodotracker.abuse.ch/
2,2,Feodo Tracker,144.139.47.206:443,ip:port,botnet,qakbot,2021-09-30,2023-08-25,Feodo Tracker blocklist,https://feodotracker.abuse.ch/
3,3,Feodo Tracker,41.86.42.158:995,ip:port,botnet,qakbot,2021-10-08,2023-08-25,Feodo Tracker blocklist,https://feodotracker.abuse.ch/
4,4,Feodo Tracker,63.143.92.99:995,ip:port,botnet,qakbot,2021-10-08,2023-08-25,Feodo Tracker blocklist,https://feodotracker.abuse.ch/
...,...,...,...,...,...,...,...,...,...,...
15580,2704832,URL Haus,http://219.157.141.108:59808/Mozi.m,url,payload delivery,NaN,2023-08-16,2023-08-16,Gandylyan1,https://urlhaus.abuse.ch/url/2704832/
15581,2704830,URL Haus,http://175.107.13.143:38812/Mozi.m,url,payload delivery,NaN,2023-08-16,2023-08-16,Gandylyan1,https://urlhaus.abuse.ch/url/2704830/
15582,2704831,URL Haus,http://102.33.46.237:54730/Mozi.m,url,payload delivery,NaN,2023-08-16,2023-08-16,Gandylyan1,https://urlhaus.abuse.ch/url/2704831/
15583,2704829,URL Haus,http://158.255.82.182:48683/Mozi.m,url,payload delivery,NaN,2023-08-16,2023-08-18,Gandylyan1,https://urlhaus.abuse.ch/url/2704829/


In [4]:
completeness_old = entries_df['malware'].count() / len(entries_df) * 100

# Display the completeness
print(f"Completeness of the 'malware' column: {completeness_old:.4f}%")


Completeness of the 'malware' column: 54.3022%


### Malwares

In [5]:
malwares_df = pd.read_csv(malwares_csv, low_memory = False)
malwares_df = malwares_df.drop('Unnamed: 0', axis = 1)

malwares_df.head(50)

,malware,type
0,qakbot,unknown
1,pikabot,unknown
2,cobalt strike,unknown
3,nanocore rat,unknown
4,remcos,unknown
5,loki password stealer pws,unknown
6,dcrat,unknown
7,redline stealer,unknown
8,meterpreter,unknown
9,lumma stealer,unknown


### Tags

In [6]:
tags_df = pd.read_csv(tags_csv, low_memory = False)
tags_df = tags_df.drop('Unnamed: 0', axis = 1)

tags_df

,ID_ENTRY,source,tag
0,2711900,URL Haus,hajime
1,2711899,URL Haus,socgholish
2,2711898,URL Haus,mozi
3,2711897,URL Haus,dropped by smokeloader
4,2711896,URL Haus,nanocore
...,...,...,...
31547,1163859,Threat Fox,rat
31548,1163860,Threat Fox,rat
31549,1163860,Threat Fox,nanocore
31550,1163861,Threat Fox,remcosrat


### Alias

In [7]:
alias_df = pd.read_csv(alias_csv, low_memory = False)
alias_df = alias_df.drop('Unnamed: 0', axis = 1)

alias_df.head(50)

,malware,alias
0,cobalt strike,agentemis
1,nanocore rat,nancrat
2,remcos,remcosrat
3,loki password stealer pws,burkina
4,dcrat,darkcrystal rat
5,lumma stealer,lummac2 stealer
6,havoc,havokiz
7,alien,alienbot
8,crimson rat,seedoor
9,agent tesla,agentesla


In [8]:
malwares_list = list(set(alias_df['malware']))

for i in range(len(malwares_list)):
    
    data = {
        'malware' : malwares_list[i], 'alias' : malwares_list[i]
    }
    df_1 = pd.DataFrame([data])
    
    alias_df = pd.concat([alias_df, df_1], ignore_index=True)
    
malwares_list = list(set(malwares_df['malware']))    
for i in range(len(malwares_list)):
    
    data = {
        'malware' : malwares_list[i], 'alias' : malwares_list[i]
    }
    df_1 = pd.DataFrame([data])
    
    alias_df = pd.concat([alias_df, df_1], ignore_index=True)    

alias_df

,malware,alias
0,cobalt strike,agentemis
1,nanocore rat,nancrat
2,remcos,remcosrat
3,loki password stealer pws,burkina
4,dcrat,darkcrystal rat
...,...,...
176,unknown malware,unknown malware
177,bitter rat,bitter rat
178,mystic stealer,mystic stealer
179,loda,loda


In [9]:
alias_df = alias_df.drop_duplicates().reset_index(drop = True)
alias_df 

,malware,alias
0,cobalt strike,agentemis
1,nanocore rat,nancrat
2,remcos,remcosrat
3,loki password stealer pws,burkina
4,dcrat,darkcrystal rat
...,...,...
140,pikabot,pikabot
141,unknown malware,unknown malware
142,bitter rat,bitter rat
143,mystic stealer,mystic stealer


In [10]:
alias_df[alias_df.malware == 'qakbot']

,malware,alias
120,qakbot,qakbot


# Data Enrichment



### Prepare the words 

In Tags could be hidden a Malware or Alias. So we search a word for similarity.

In [11]:
malware_list = list(set(alias_df['malware']))
alias_list   = list(set(alias_df['alias']))
tag_list     = list(set(tags_df['tag']))

malware_words = list(set(malware_list + alias_list))

In [12]:
len(tag_list)

701

In [13]:
len(malware_list)

70

In [14]:
len(alias_list)

144

In [15]:
len(malware_words)

144

#### Jaro-Wrinkler Similarity index

In [16]:
# Jaro Winkler Similarity
def jaro_Winkler(s1, s2) :
 
    jaro_dist = jaro_distance(s1, s2);
 
    # If the jaro Similarity is above a threshold
    if (jaro_dist > 0.7) :
 
        # Find the length of common prefix
        prefix = 0;
 
        for i in range(min(len(s1), len(s2))) :
         
            # If the characters match
            if (s1[i] == s2[i]) :
                prefix += 1;
 
            # Else break
            else :
                break;
 
        # Maximum of 4 characters are allowed in prefix
        prefix = min(4, prefix);
 
        # Calculate jaro winkler Similarity
        jaro_dist += 0.1 * prefix * (1 - jaro_dist);
 
    return jaro_dist;


def jaro_distance(s1, s2) :
 
    # If the strings are equal
    if (s1 == s2) :
        return 1.0;
 
    # Length of two strings
    len1 = len(s1);
    len2 = len(s2);
 
    if (len1 == 0 or len2 == 0) :
        return 0.0;
 
    # Maximum distance upto which matching
    # is allowed
    max_dist = (max(len(s1), len(s2)) // 2 ) - 1;
 
    # Count of matches
    match = 0;
 
    # Hash for matches
    hash_s1 = [0] * len(s1) ;
    hash_s2 = [0] * len(s2) ;
 
    # Traverse through the first string
    for i in range(len1) :
 
        # Check if there is any matches
        for j in range( max(0, i - max_dist),
                    min(len2, i + max_dist + 1)) :
             
            # If there is a match
            if (s1[i] == s2[j] and hash_s2[j] == 0) :
                hash_s1[i] = 1;
                hash_s2[j] = 1;
                match += 1;
                break;
         
    # If there is no match
    if (match == 0) :
        return 0.0;
 
    # Number of transpositions
    t = 0;
 
    point = 0;
 
    # Count number of occurrences
    # where two characters match but
    # there is a third matched character
    # in between the indices
    for i in range(len1) :
        if (hash_s1[i]) :
 
            # Find the next matched character
            # in second string
            while (hash_s2[point] == 0) :
                point += 1;
 
            if (s1[i] != s2[point]) :
                point += 1;
                t += 1;
            else :
                point += 1;
                 
        t /= 2;
 
    # Return the Jaro Similarity
    return ((match / len1 + match / len2 +
            (match - t) / match ) / 3.0);

In [17]:
def compute_matrix(lista_riga, lista_colonna):
    
    df = pd.DataFrame()
    
    df.index = lista_riga
    
    df.rename(columns=lista_colonna, inplace=True)  
    
    for i in range(len(lista_riga)):
        for j in range(len(lista_colonna)):
            
            string1 = lista_riga[i]
            string2 = lista_colonna[j]
            
            jaro_wrinkler = jaro_Winkler(string1,string2)
            
            df.loc[string1, string2] = jaro_wrinkler
            
    return df
            
            

In [18]:
jaro_wrinkler_matrix = compute_matrix( tag_list, malware_words)
jaro_wrinkler_matrix

C:\Users\leona\AppData\Local\Temp\ipykernel_5984\2389614628.py:17: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead. To get a de-fragmented frame, use `newframe = frame.copy()`
  df.loc[string1, string2] = jaro_wrinkler
C:\Users\leona\AppData\Local\Temp\ipykernel_5984\2389614628.py:17: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead. To get a de-fragmented frame, use `newframe = frame.copy()`
  df.loc[string1, string2] = jaro_wrinkler
C:\Users\leona\AppData\Local\Temp\ipykernel_5984\2389614628.py:17: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joi

,tvrat,icedid,dinihou,darkcomet,agentemis,spiderlabs responder,coldtrain,guildma,alien,hydra,...,arechclient,smoke loader,azorult,vidar,negasteal,tvspy,pikabot,kognito,bitter rat,cobeacon
agenttesla dropper,0.588882,0.481481,0.465608,0.552945,0.833333,0.733515,0.665308,0.597150,0.673989,0.418519,...,0.626190,0.748056,0.531631,0.503704,0.833193,0.503704,0.528267,0.597884,0.643565,0.571466
smartapesg,0.631163,0.000000,0.000000,0.603096,0.614815,0.622266,0.542274,0.495238,0.533333,0.531380,...,0.524242,0.564757,0.576190,0.533333,0.674443,0.631489,0.495238,0.414286,0.585026,0.461849
discord infostealer pw: epsilon beta,0.561111,0.722222,0.617725,0.657407,0.657407,0.774050,0.750000,0.560846,0.712963,0.561111,...,0.689394,0.703704,0.617725,0.637037,0.657407,0.561111,0.560847,0.617725,0.631481,0.638889
dropped by privateloader encrypted redline sectoprat,0.625641,0.581197,0.549451,0.637464,0.594017,0.725641,0.680912,0.549451,0.698718,0.625641,...,0.663753,0.641026,0.603480,0.698718,0.594017,0.625641,0.657509,0.603480,0.691026,0.669872
evoxtenterprise as ap evoxt enterprise,0.710526,0.461988,0.502506,0.562378,0.653996,0.714912,0.608187,0.446115,0.635088,0.484211,...,0.606858,0.625728,0.558897,0.635088,0.653996,0.635088,0.615288,0.615288,0.712281,0.585522
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
kpn kpn national,0.508333,0.409722,0.470238,0.498264,0.506944,0.610684,0.560903,0.401786,0.420833,0.420833,...,0.481527,0.521918,0.401786,0.420833,0.622685,0.420833,0.670384,0.672470,0.493216,0.513014
shenzhen tencent computer systems company limited,0.553741,0.458050,0.551020,0.596372,0.596372,0.614959,0.552532,0.442177,0.480272,0.480272,...,0.630179,0.575397,0.605442,0.406803,0.552532,0.627211,0.496599,0.496599,0.574150,0.575680
darkgate pw 2233,0.595721,0.485783,0.401786,0.680393,0.564571,0.557568,0.564731,0.538658,0.508333,0.595823,...,0.537635,0.527277,0.538690,0.595833,0.622553,0.508333,0.538674,0.538690,0.603705,0.458003
amadey dropped by privateloader smoke loader,0.630303,0.585859,0.498918,0.645623,0.601010,0.745449,0.601010,0.554113,0.556061,0.630303,...,0.598485,0.686868,0.609307,0.704545,0.601010,0.556061,0.664502,0.554113,0.660606,0.579545


In [19]:
def search_similarity(df, alpha):
    
    dataframe = pd.DataFrame()
    
    indexes = list(df.index)
    columns = list(df.columns)
    
    for i in range(len(indexes)):
        for j in range(len(columns)):
            
            x = indexes[i]
            y = columns[j]
                        
            element = df.at[x, y]
            
            if((element > alpha)):
#                print(f'x {x} and y: {y} and elem : {element}')
                data = {'malware_OR_alias': [y], 'tag': [x]}
                temp = pd.DataFrame(data)
                
                dataframe = pd.concat([dataframe, temp], ignore_index= True)
    
    return dataframe

In [20]:
similarity_dataframe = search_similarity(jaro_wrinkler_matrix, 0.8)
similarity_dataframe = similarity_dataframe.drop_duplicates()
print(f'Matches between alias, malware and tag: {len(similarity_dataframe)}')
similarity_dataframe.head(50)

Matches between alias, malware and tag: 471


,malware_OR_alias,tag
0,agentemis,agenttesla dropper
1,agenttesla,agenttesla dropper
2,agentesla,agenttesla dropper
3,agent tesla,agenttesla dropper
4,negasteal,agenttesla dropper
5,mars stealer,smartapesg
6,loki password stealer pws,discord infostealer pw: epsilon beta
7,xworm,xtom
8,puffstealer,stealer
9,stealc,stealer


#### new table TAGS : add Associated Malware column

In [21]:
new_tags_df = tags_df.copy()

new_tags_df['Ass.Malware'] = ''
new_tags_df

,ID_ENTRY,source,tag,Ass.Malware
0,2711900,URL Haus,hajime,
1,2711899,URL Haus,socgholish,
2,2711898,URL Haus,mozi,
3,2711897,URL Haus,dropped by smokeloader,
4,2711896,URL Haus,nanocore,
...,...,...,...,...
31547,1163859,Threat Fox,rat,
31548,1163860,Threat Fox,rat,
31549,1163860,Threat Fox,nanocore,
31550,1163861,Threat Fox,remcosrat,


In [22]:
def add_associated_malware(tags, similarities, alias):
    
    malwares = list(set(alias.malware))
    
    for i in range(len(tags)):
        
        tag = tags.at[i, 'tag']
        similarity_df = similarities[similarities.tag == tag].reset_index(drop = True)
        similarity_list = list(similarity_df['malware_OR_alias'])
        
        if(len(similarity_list) > 0):            
            dataframe = pd.DataFrame()
            
            for j in range(len(similarity_list)):
                
                similarity_element = similarity_list[j]
                match_df = alias[alias.alias == similarity_element].reset_index(drop = True)
                dataframe = pd.concat([dataframe, match_df])
                
            malware = list(set(dataframe['malware']))
            
            if(len(malware) == 1):
                #print(f'Found in {tag} : {similarity_list} --> Prediction : {malware[0]}')
            
                tags.at[i, 'Ass.Malware'] = malware[0]       
                            
    return tags

In [23]:
new_tags_df = add_associated_malware(new_tags_df, similarity_dataframe, alias_df)
new_tags_df

,ID_ENTRY,source,tag,Ass.Malware
0,2711900,URL Haus,hajime,
1,2711899,URL Haus,socgholish,fakeupdates
2,2711898,URL Haus,mozi,mozi
3,2711897,URL Haus,dropped by smokeloader,
4,2711896,URL Haus,nanocore,nanocore rat
...,...,...,...,...
31547,1163859,Threat Fox,rat,irata
31548,1163860,Threat Fox,rat,irata
31549,1163860,Threat Fox,nanocore,nanocore rat
31550,1163861,Threat Fox,remcosrat,


In [24]:
new_tags_df[(new_tags_df['Ass.Malware'] != '') & (new_tags_df['ID_ENTRY'] == 2711899)].head(50)

,ID_ENTRY,source,tag,Ass.Malware
1,2711899,URL Haus,socgholish,fakeupdates


### Enrich Malware field in Entries dataframe with Majority Voting 

In [25]:
from collections import Counter

def majority_vote(lst):
    # Count occurrences of each element in the list
    counts = Counter(lst)

    # Find the element with the maximum count
    most_common_element = max(counts, key=counts.get)
    
    return most_common_element

def assign_malware(entries, new_tags) :
    
    for i in range(len(entries)):
        
        malware  = entries.at[i, 'malware']
        id_entry = entries.at[i, 'ID_ENTRY']
        source   = entries.at[i, 'source']
                
        if not isinstance(malware, str) :
            
            malwares = list(new_tags_df[( new_tags_df['Ass.Malware'] != '' ) & ( new_tags.ID_ENTRY == id_entry ) &  ( new_tags.source == source )]['Ass.Malware'])
            if(len(malwares) >= 1):
                entries.at[i, 'malware'] = majority_vote(malwares)
            
    return entries            

In [26]:
entries_assigned = assign_malware(entries_df, new_tags_df)
entries_assigned

,ID_ENTRY,source,ioc,ioc_type,threat_type,malware,first_seen,last_seen,reporter,reference
0,0,Feodo Tracker,89.101.97.139:443,ip:port,botnet,qakbot,2021-09-29,2023-08-25,Feodo Tracker blocklist,https://feodotracker.abuse.ch/
1,1,Feodo Tracker,41.228.22.180:443,ip:port,botnet,qakbot,2021-09-29,2023-08-25,Feodo Tracker blocklist,https://feodotracker.abuse.ch/
2,2,Feodo Tracker,144.139.47.206:443,ip:port,botnet,qakbot,2021-09-30,2023-08-25,Feodo Tracker blocklist,https://feodotracker.abuse.ch/
3,3,Feodo Tracker,41.86.42.158:995,ip:port,botnet,qakbot,2021-10-08,2023-08-25,Feodo Tracker blocklist,https://feodotracker.abuse.ch/
4,4,Feodo Tracker,63.143.92.99:995,ip:port,botnet,qakbot,2021-10-08,2023-08-25,Feodo Tracker blocklist,https://feodotracker.abuse.ch/
...,...,...,...,...,...,...,...,...,...,...
15580,2704832,URL Haus,http://219.157.141.108:59808/Mozi.m,url,payload delivery,mozi,2023-08-16,2023-08-16,Gandylyan1,https://urlhaus.abuse.ch/url/2704832/
15581,2704830,URL Haus,http://175.107.13.143:38812/Mozi.m,url,payload delivery,mozi,2023-08-16,2023-08-16,Gandylyan1,https://urlhaus.abuse.ch/url/2704830/
15582,2704831,URL Haus,http://102.33.46.237:54730/Mozi.m,url,payload delivery,mozi,2023-08-16,2023-08-16,Gandylyan1,https://urlhaus.abuse.ch/url/2704831/
15583,2704829,URL Haus,http://158.255.82.182:48683/Mozi.m,url,payload delivery,mozi,2023-08-16,2023-08-18,Gandylyan1,https://urlhaus.abuse.ch/url/2704829/


In [27]:
enriched_entries = entries_assigned
enriched_entries

,ID_ENTRY,source,ioc,ioc_type,threat_type,malware,first_seen,last_seen,reporter,reference
0,0,Feodo Tracker,89.101.97.139:443,ip:port,botnet,qakbot,2021-09-29,2023-08-25,Feodo Tracker blocklist,https://feodotracker.abuse.ch/
1,1,Feodo Tracker,41.228.22.180:443,ip:port,botnet,qakbot,2021-09-29,2023-08-25,Feodo Tracker blocklist,https://feodotracker.abuse.ch/
2,2,Feodo Tracker,144.139.47.206:443,ip:port,botnet,qakbot,2021-09-30,2023-08-25,Feodo Tracker blocklist,https://feodotracker.abuse.ch/
3,3,Feodo Tracker,41.86.42.158:995,ip:port,botnet,qakbot,2021-10-08,2023-08-25,Feodo Tracker blocklist,https://feodotracker.abuse.ch/
4,4,Feodo Tracker,63.143.92.99:995,ip:port,botnet,qakbot,2021-10-08,2023-08-25,Feodo Tracker blocklist,https://feodotracker.abuse.ch/
...,...,...,...,...,...,...,...,...,...,...
15580,2704832,URL Haus,http://219.157.141.108:59808/Mozi.m,url,payload delivery,mozi,2023-08-16,2023-08-16,Gandylyan1,https://urlhaus.abuse.ch/url/2704832/
15581,2704830,URL Haus,http://175.107.13.143:38812/Mozi.m,url,payload delivery,mozi,2023-08-16,2023-08-16,Gandylyan1,https://urlhaus.abuse.ch/url/2704830/
15582,2704831,URL Haus,http://102.33.46.237:54730/Mozi.m,url,payload delivery,mozi,2023-08-16,2023-08-16,Gandylyan1,https://urlhaus.abuse.ch/url/2704831/
15583,2704829,URL Haus,http://158.255.82.182:48683/Mozi.m,url,payload delivery,mozi,2023-08-16,2023-08-18,Gandylyan1,https://urlhaus.abuse.ch/url/2704829/


In [28]:
completeness_new = enriched_entries['malware'].count() / len(enriched_entries) * 100
print(f"Completeness of the 'malware' column: {completeness_new:.4f}%")

Completeness of the 'malware' column: 63.8819%


In [29]:
enriched_entries.loc[enriched_entries.malware.isna(),'malware'] = 'unknown'
enriched_entries

,ID_ENTRY,source,ioc,ioc_type,threat_type,malware,first_seen,last_seen,reporter,reference
0,0,Feodo Tracker,89.101.97.139:443,ip:port,botnet,qakbot,2021-09-29,2023-08-25,Feodo Tracker blocklist,https://feodotracker.abuse.ch/
1,1,Feodo Tracker,41.228.22.180:443,ip:port,botnet,qakbot,2021-09-29,2023-08-25,Feodo Tracker blocklist,https://feodotracker.abuse.ch/
2,2,Feodo Tracker,144.139.47.206:443,ip:port,botnet,qakbot,2021-09-30,2023-08-25,Feodo Tracker blocklist,https://feodotracker.abuse.ch/
3,3,Feodo Tracker,41.86.42.158:995,ip:port,botnet,qakbot,2021-10-08,2023-08-25,Feodo Tracker blocklist,https://feodotracker.abuse.ch/
4,4,Feodo Tracker,63.143.92.99:995,ip:port,botnet,qakbot,2021-10-08,2023-08-25,Feodo Tracker blocklist,https://feodotracker.abuse.ch/
...,...,...,...,...,...,...,...,...,...,...
15580,2704832,URL Haus,http://219.157.141.108:59808/Mozi.m,url,payload delivery,mozi,2023-08-16,2023-08-16,Gandylyan1,https://urlhaus.abuse.ch/url/2704832/
15581,2704830,URL Haus,http://175.107.13.143:38812/Mozi.m,url,payload delivery,mozi,2023-08-16,2023-08-16,Gandylyan1,https://urlhaus.abuse.ch/url/2704830/
15582,2704831,URL Haus,http://102.33.46.237:54730/Mozi.m,url,payload delivery,mozi,2023-08-16,2023-08-16,Gandylyan1,https://urlhaus.abuse.ch/url/2704831/
15583,2704829,URL Haus,http://158.255.82.182:48683/Mozi.m,url,payload delivery,mozi,2023-08-16,2023-08-18,Gandylyan1,https://urlhaus.abuse.ch/url/2704829/


Add new possible aliases to alias from the New Tag Table

### Enrich Type field in malwares dataframe with Majority Voting

In [30]:
new_alias = pd.DataFrame()
new_tags_df = new_tags_df[(new_tags_df['Ass.Malware'] != '')]
new_alias[['alias', 'malware']] = new_tags_df[['tag','Ass.Malware']]
new_alias = new_alias.drop_duplicates()
new_alias


,alias,malware
1,socgholish,fakeupdates
2,mozi,mozi
4,nanocore,nanocore rat
5,exe stealc,stealc
27,dropped by smokeloader stealc,smokeloader
...,...,...
30776,contabo,cobalt strike
30969,tango telindus,agent tesla
30978,google cloud platform,gootloader
31157,kimusky,kimsuky


In [31]:
alias_df

,malware,alias
0,cobalt strike,agentemis
1,nanocore rat,nancrat
2,remcos,remcosrat
3,loki password stealer pws,burkina
4,dcrat,darkcrystal rat
...,...,...
140,pikabot,pikabot
141,unknown malware,unknown malware
142,bitter rat,bitter rat
143,mystic stealer,mystic stealer


In [32]:
alias_updated = pd.concat([alias_df, new_alias], axis = 0)
alias_updated = alias_updated.drop_duplicates().reset_index(drop = True)
alias_updated

,malware,alias
0,cobalt strike,agentemis
1,nanocore rat,nancrat
2,remcos,remcosrat
3,loki password stealer pws,burkina
4,dcrat,darkcrystal rat
...,...,...
284,redline stealer,power line datacenter
285,cobalt strike,contabo
286,agent tesla,tango telindus
287,gootloader,google cloud platform


In [33]:
enriched_alias = alias_updated
enriched_alias

,malware,alias
0,cobalt strike,agentemis
1,nanocore rat,nancrat
2,remcos,remcosrat
3,loki password stealer pws,burkina
4,dcrat,darkcrystal rat
...,...,...
284,redline stealer,power line datacenter
285,cobalt strike,contabo
286,agent tesla,tango telindus
287,gootloader,google cloud platform


In [34]:
alias_updated[alias_updated.malware == 'qakbot']

,malware,alias
120,qakbot,qakbot


- Virus: I virus sono programmi che si replicano inserendo il proprio codice in altri programmi esistenti. Possono infettare file eseguibili o documenti di vario tipo. Sigle : VBS (Visual Basic Script), JS (JavaScript), EXE, DOC, XLS

- Worm: I worm sono programmi autonomi che si replicano e diffondono attraverso reti e sistemi, spesso sfruttando vulnerabilità di sicurezza per la propagazione automatica. Sigle: MSBlast (Blaster), Sasser, Conficker, Slammer

- Trojan: I trojan sono programmi che sembrano essere legittimi ma in realtà contengono un componente dannoso. Possono aprire una "porta posteriore" sul sistema, consentendo l'accesso non autorizzato o installando altri malware. Sigle: Troj, Trojan, TR, rat

- Ransomware: Il ransomware cifra i file della vittima e richiede un riscatto in cambio della chiave per sbloccarli. Può causare danni significativi ai dati personali o aziendali. Sigle: Ransom, Crypt, Locker

- Spyware: Il spyware monitora segretamente l'attività dell'utente, raccoglie informazioni sensibili e le invia a terze parti senza il consenso dell'utente. Sigle : Spy, Keylog, Key

- Adware: L'adware mostra pubblicità indesiderate sul computer dell'utente. Può essere integrato in software legittimo o installato separatamente. Sigle : Ad, Adw, Adware

- Rootkit: I rootkit nascondono la presenza di malware, spesso modificando o sostituendo parti del sistema operativo. Possono essere utilizzati per garantire l'accesso continuo e non autorizzato. Sigle : RK, Rootkit, Root

- Botnet: Un botnet è una rete di computer infetti (zombie) controllati da un comando centrale. Possono essere utilizzati per attacchi coordinati come DDoS o per inviare spam. Sigle : Bot, Zombie, Bnspy

- Keylogger: Un keylogger registra le tastate dell'utente senza il loro consenso. Può essere utilizzato per raccogliere informazioni come password e dati finanziari. Sigle :  KL, Keylog, Klog

- Scareware: Lo scareware tenta di spaventare gli utenti mostrando avvisi falsi o messaggi di minaccia per indurli a acquistare software inutile o dannoso. Sigle : FakeAV, Scare, Scareware

- Logic Bomb: Una bomba logica è un tipo di malware che viene attivato quando si verifica una condizione specifica, ad esempio una certa data o evento. Sigle : LB, Logic, Bomb

- Backdoor: Una backdoor fornisce un accesso segreto al sistema, consentendo a un attaccante di controllare il computer senza il consenso dell'utente. Sigle : Bkdr, Backdoor, BD

- Dropper: Un dropper è progettato per "rilasciare" o installare altri malware sul sistema dell'utente. Sigle : Dropper, Drop, Dldr

- Downloader: Un downloader è progettato per scaricare e installare altri malware o componenti dannosi sul sistema dell'utente. Sigle : Down, Downloader, DL

- Exploit: Un exploit sfrutta vulnerabilità di sicurezza nel software o nel sistema operativo per eseguire codice dannoso. Sigle : Exp, Exploit

- Grayware: Il grayware è software potenzialmente indesiderato che potrebbe non essere categorizzato come malware, ma può comunque comportarsi in modo irritante o invasivo. Include spesso adware e PUP (Potentially Unwanted Programs). Sigle : PUP, Grayware


In [35]:
alias_categorized = alias_updated.copy()

alias_categorized['worm'] = 0
alias_categorized['trojan'] = 0
alias_categorized['ransomware'] = 0
alias_categorized['rootkit'] = 0
alias_categorized['spyware'] = 0
alias_categorized['adware'] = 0
alias_categorized['botnet'] = 0
alias_categorized['keylogger'] = 0
alias_categorized['dropper'] = 0
alias_categorized['backdoor'] = 0
alias_categorized['downloader'] = 0


worm_keys       = ['worm', 'w0rm', 'slam']
trojan_keys     = ['rat', 'trj', 'troj']
ransomware_keys = ['ransom', 'crypt', 'lock']
keylog_keys     = ['key', 'keylog', ]
adware_keys     = ['adw']
rootkit_keys    = ['root', 'kit']
botnet_keys     = ['bot', 'c2', 'cc']
spyware_keys    = ['spy', 'steal']
backdoor_keys   = ['bd', 'back', 'door']
download_keys   = ['down', 'dl', 'load']
dropper_keys    = ['drop', 'drp']


def verifica_presenza(parole_chiave, stringa):
    for parola_chiave in parole_chiave:
        if parola_chiave.lower() in stringa.lower():
            return True
    return False

for i in range(len(alias_categorized)):
    
    tag_name = alias_categorized.at[i, 'alias']
    
    if verifica_presenza(worm_keys, tag_name):
        alias_categorized.loc[i, 'worm'] = 1
        
    if verifica_presenza(trojan_keys, tag_name):
        alias_categorized.loc[i, 'trojan'] = 1  
        
    if verifica_presenza(ransomware_keys, tag_name):
        alias_categorized.loc[i, 'ransomware'] = 1
        
    if verifica_presenza(rootkit_keys, tag_name):
        alias_categorized.loc[i, 'rootkit'] = 1
        
    if verifica_presenza(spyware_keys, tag_name):
        alias_categorized.loc[i, 'spyware'] = 1  
        
    if verifica_presenza(adware_keys, tag_name):
        alias_categorized.loc[i, 'adware'] = 1
        
    if verifica_presenza(botnet_keys, tag_name):
        alias_categorized.loc[i, 'botnet'] = 1
        
    if verifica_presenza(keylog_keys, tag_name):
        alias_categorized.loc[i, 'keylogger'] = 1    
        
    if verifica_presenza(dropper_keys, tag_name):
        alias_categorized.loc[i, 'dropper'] = 1    
        
    if verifica_presenza(backdoor_keys, tag_name):
        alias_categorized.loc[i, 'backdoor'] = 1   
        
    if verifica_presenza(download_keys, tag_name):
        alias_categorized.loc[i, 'downloader'] = 1 
        
alias_categorized       


,malware,alias,worm,trojan,ransomware,rootkit,spyware,adware,botnet,keylogger,dropper,backdoor,downloader
0,cobalt strike,agentemis,0,0,0,0,0,0,0,0,0,0,0
1,nanocore rat,nancrat,0,1,0,0,0,0,0,0,0,0,0
2,remcos,remcosrat,0,1,0,0,0,0,0,0,0,0,0
3,loki password stealer pws,burkina,0,0,0,0,0,0,0,0,0,0,0
4,dcrat,darkcrystal rat,0,1,0,0,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...
284,redline stealer,power line datacenter,0,0,0,0,0,0,0,0,0,0,0
285,cobalt strike,contabo,0,0,0,0,0,0,0,0,0,0,0
286,agent tesla,tango telindus,0,0,0,0,0,0,0,0,0,0,0
287,gootloader,google cloud platform,0,0,0,0,0,0,0,0,0,0,0


In [36]:
alias_categorized.columns

Index(['malware', 'alias', 'worm', 'trojan', 'ransomware', 'rootkit',
       'spyware', 'adware', 'botnet', 'keylogger', 'dropper', 'backdoor',
       'downloader'],
      dtype='object')

In [37]:
grouped_df = alias_categorized[['malware', 'worm', 'trojan', 'ransomware', 'rootkit',
       'spyware', 'adware', 'botnet', 'keylogger', 'dropper', 'backdoor',
       'downloader']].groupby('malware').sum()
max_column = grouped_df.idxmax(axis=1)

# Se la somma è zero, assegna 'unknown'
max_column[grouped_df.sum(axis=1) == 0] = 'unknown'
type_df = pd.DataFrame(max_column).reset_index()
type_df

,malware,0
0,agent tesla,trojan
1,alien,botnet
2,amadey,dropper
3,astaroth,unknown
4,asyncrat,trojan
...,...,...
65,unknown malware,unknown
66,vidar,backdoor
67,vjw0rm,worm
68,whitesnake stealer,spyware


In [38]:
for i in range(len(type_df)):
    
    m = type_df.at[i, 'malware']
    t = type_df.at[i, 0]
    
    malwares_df.loc[malwares_df['malware'] == m, 'type'] = t

malwares_df.head(50)

,malware,type
0,qakbot,botnet
1,pikabot,botnet
2,cobalt strike,unknown
3,nanocore rat,trojan
4,remcos,trojan
5,loki password stealer pws,spyware
6,dcrat,trojan
7,redline stealer,spyware
8,meterpreter,unknown
9,lumma stealer,spyware


In [39]:
enriched_malwares = malwares_df
enriched_malwares

,malware,type
0,qakbot,botnet
1,pikabot,botnet
2,cobalt strike,unknown
3,nanocore rat,trojan
4,remcos,trojan
...,...,...
65,loda,trojan
66,spynote,trojan
67,sectoprat,trojan
68,ghost rat,trojan


### Enrich Type field in tags dataframe with Majority Voting

In [40]:
old_tags = tags_df.copy()

old_tags['worm'] = 0
old_tags['trojan'] = 0
old_tags['ransomware'] = 0
old_tags['rootkit'] = 0
old_tags['spyware'] = 0
old_tags['adware'] = 0
old_tags['botnet'] = 0
old_tags['keylogger'] = 0
old_tags['dropper'] = 0
old_tags['backdoor'] = 0
old_tags['downloader'] = 0


worm_keys       = ['worm', 'w0rm', 'slam']
trojan_keys     = ['rat', 'trj', 'troj']
ransomware_keys = ['ransom', 'crypt', 'lock']
keylog_keys     = ['key', 'keylog', ]
adware_keys     = ['adw']
rootkit_keys    = ['root', 'kit']
botnet_keys     = ['bot', 'c2', 'cc']
spyware_keys    = ['spy', 'steal']
backdoor_keys   = ['bd', 'back', 'door']
download_keys   = ['down', 'dl', 'load']
dropper_keys    = ['drop', 'drp']


def verifica_presenza(parole_chiave, stringa):
    for parola_chiave in parole_chiave:
        if parola_chiave.lower() in stringa.lower():
            return True
    return False

for i in range(len(old_tags)):
    
    tag_name = old_tags.at[i, 'tag']
    
    if verifica_presenza(worm_keys, tag_name):
        old_tags.loc[i, 'worm'] = 1
        
    if verifica_presenza(trojan_keys, tag_name):
        old_tags.loc[i, 'trojan'] = 1  
        
    if verifica_presenza(ransomware_keys, tag_name):
        old_tags.loc[i, 'ransomware'] = 1
        
    if verifica_presenza(rootkit_keys, tag_name):
        old_tags.loc[i, 'rootkit'] = 1
        
    if verifica_presenza(spyware_keys, tag_name):
        old_tags.loc[i, 'spyware'] = 1  
        
    if verifica_presenza(adware_keys, tag_name):
        old_tags.loc[i, 'adware'] = 1
        
    if verifica_presenza(botnet_keys, tag_name):
        old_tags.loc[i, 'botnet'] = 1
        
    if verifica_presenza(keylog_keys, tag_name):
        old_tags.loc[i, 'keylogger'] = 1    
        
    if verifica_presenza(dropper_keys, tag_name):
        old_tags.loc[i, 'dropper'] = 1    
        
    if verifica_presenza(backdoor_keys, tag_name):
        old_tags.loc[i, 'backdoor'] = 1   
        
    if verifica_presenza(download_keys, tag_name):
        old_tags.loc[i, 'downloader'] = 1 
        
old_tags
        
        

,ID_ENTRY,source,tag,worm,trojan,ransomware,rootkit,spyware,adware,botnet,keylogger,dropper,backdoor,downloader
0,2711900,URL Haus,hajime,0,0,0,0,0,0,0,0,0,0,0
1,2711899,URL Haus,socgholish,0,0,0,0,0,0,0,0,0,0,0
2,2711898,URL Haus,mozi,0,0,0,0,0,0,0,0,0,0,0
3,2711897,URL Haus,dropped by smokeloader,0,0,0,0,0,0,0,0,1,0,1
4,2711896,URL Haus,nanocore,0,0,0,0,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
31547,1163859,Threat Fox,rat,0,1,0,0,0,0,0,0,0,0,0
31548,1163860,Threat Fox,rat,0,1,0,0,0,0,0,0,0,0,0
31549,1163860,Threat Fox,nanocore,0,0,0,0,0,0,0,0,0,0,0
31550,1163861,Threat Fox,remcosrat,0,1,0,0,0,0,0,0,0,0,0


In [41]:
old_tags.iloc[:, 3:-1].max(axis=1)

0        0
1        0
2        0
3        1
4        0
        ..
31547    1
31548    1
31549    0
31550    1
31551    1
Length: 31552, dtype: int64

In [42]:
max_greater_than_zero = old_tags.iloc[:, 3:-1].max(axis=1) >= 1
max_greater_than_zero

0        False
1        False
2        False
3         True
4        False
         ...  
31547     True
31548     True
31549    False
31550     True
31551     True
Length: 31552, dtype: bool

In [43]:
colonna_maggiore = old_tags.iloc[:, 3:-1].idxmax(axis=1)
colonna_maggiore

0           worm
1           worm
2           worm
3        dropper
4           worm
          ...   
31547     trojan
31548     trojan
31549       worm
31550     trojan
31551     trojan
Length: 31552, dtype: object

In [44]:
old_tags['type'] = ''
colonna_maggiore = old_tags.iloc[:, 3:-1].idxmax(axis=1)
max_greater_than_zero = old_tags.iloc[:, 3:-1].max(axis=1) == 0
old_tags['type'] = colonna_maggiore

old_tags.loc[max_greater_than_zero, 'type'] = 'unknown'

old_tags

,ID_ENTRY,source,tag,worm,trojan,ransomware,rootkit,spyware,adware,botnet,keylogger,dropper,backdoor,downloader,type
0,2711900,URL Haus,hajime,0,0,0,0,0,0,0,0,0,0,0,unknown
1,2711899,URL Haus,socgholish,0,0,0,0,0,0,0,0,0,0,0,unknown
2,2711898,URL Haus,mozi,0,0,0,0,0,0,0,0,0,0,0,unknown
3,2711897,URL Haus,dropped by smokeloader,0,0,0,0,0,0,0,0,1,0,1,dropper
4,2711896,URL Haus,nanocore,0,0,0,0,0,0,0,0,0,0,0,unknown
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
31547,1163859,Threat Fox,rat,0,1,0,0,0,0,0,0,0,0,0,trojan
31548,1163860,Threat Fox,rat,0,1,0,0,0,0,0,0,0,0,0,trojan
31549,1163860,Threat Fox,nanocore,0,0,0,0,0,0,0,0,0,0,0,unknown
31550,1163861,Threat Fox,remcosrat,0,1,0,0,0,0,0,0,0,0,0,trojan


In [45]:
old_tags.type.unique()

array(['unknown', 'dropper', 'spyware', 'trojan', 'ransomware',
       'downloader', 'botnet', 'worm', 'keylogger', 'backdoor', 'rootkit'],
      dtype=object)

In [46]:
tag_columns = ['ID_ENTRY', 'source', 'tag', 'type']
enriched_tags = old_tags[tag_columns]

In [47]:
enriched_tags.head(50)

,ID_ENTRY,source,tag,type
0,2711900,URL Haus,hajime,unknown
1,2711899,URL Haus,socgholish,unknown
2,2711898,URL Haus,mozi,unknown
3,2711897,URL Haus,dropped by smokeloader,dropper
4,2711896,URL Haus,nanocore,unknown
5,2711895,URL Haus,exe stealc,spyware
6,2711894,URL Haus,hajime,unknown
7,2711893,URL Haus,exe stealc,spyware
8,2711892,URL Haus,exe fabookie,unknown
9,2711891,URL Haus,elf mozi,unknown


#### Save into ..\IntegratedDataset\PostDataEnrichment

In [48]:
dataframes_dict = {}

# Add the DataFrames to the dictionary with keys
dataframes_dict['Entries']   = enriched_entries
dataframes_dict['Tags']      = enriched_tags
dataframes_dict['Malwares']  = enriched_malwares
dataframes_dict['Alias']     = enriched_alias


In [49]:
save_directory = "C:\\Users\\leona\\OneDrive\\Desktop\\Tesi\\Pipeline\\Datasets\\Pipeline\\IocSegnalations\\IntegratedDataset\\PostDataEnrichment\\"
for key, df in dataframes_dict.items():
    file_name = f"{key}.csv"
    file_path = save_directory + file_name
    df.to_csv(file_path, index=False)
    print(f'{file_name} saved')

Entries.csv saved
Tags.csv saved
Malwares.csv saved
Alias.csv saved
